In [1]:
import os
import sys
import json
import time
import math
import gspread
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from scipy import integrate, signal, stats, fftpack
from glob import glob
from datetime import date
# import mne

In [2]:
sys.path.append(r'C:\Users\lesliec\code')

In [3]:
from tbd_eeg.tbd_eeg.data_analysis.eegutils import EEGexp
from tbd_eeg.tbd_eeg.data_analysis.Utilities.utilities import get_stim_events, get_evoked_traces, find_nearest_ind

from PCIst.PCIst.pci_st import calc_PCIst, dimensionality_reduction

In [4]:
%matplotlib notebook

#### Load Zap_Zip-log_exp to get metadata for experiments

In [5]:
_gc = gspread.service_account() # need a key file to access the account
_sh = _gc.open('Zap_Zip-log_exp') # open the spreadsheet
_df = pd.DataFrame(_sh.sheet1.get()) # load the first worksheet
zzmetadata = _df.T.set_index(0).T # put it in a nicely formatted dataframe

In [6]:
zzmetadata.head()

,mouse_name,exp_name,brain states,stimulation,visual_stim,audio_stim,ISI (sec),stimulus duration (msec),Current (uA),Cortical Area stimulation,N trials per stimulus,EEG bad_channels,Npx,Units Sorted (X),Brain slices (X),Brain areas assignment,"CCF coordinates stim electrode (surface,tip)","CCF area stim electrode (surface,tip)",Pupil processed,Notes
1,mouse496220,audio_vis1_2020-06-10_14-54-43,awake/ISO,sensory,black/white,whitenoise/10000,5,250,,,60,None,None,None,None,None,None,None,None,None
2,mouse496220,audio_vis2_2020-06-11_11-42-47,awake/ISO,sensory,black/white,whitenoise/10000,5,250,,,60,29,None,None,None,None,None,None,None,None
3,mouse496220,audio_vis3_2020-06-16_10-35-57,run/resting,sensory,black,whitenoise,5,250,,,20,None,None,None,None,None,None,None,None,None
4,mouse496220,audio_vis4_2020-06-18_13-49-17,run/resting,sensory,black/white,whitenoise/10000,5,250,,,60,None,None,None,None,None,None,None,None,None
5,mouse521885,audio_vis1_2020-07-08_12-37-58,awake/ISO,sensory,black/white,whitenoise/10000,[3.5 4.5],250,,,50,6,None,None,None,None,None,None,None,None


### Master list of subjects for zap&zip 1st manuscript (iso data)

In [7]:
all_subjects = {
    'MOs_superficial': {
        '521885': {
            'data_loc': r'C:\Users\lesliec\OneDrive - Allen Institute\data\mouse521885\estim1_2020-07-09_14-23-49\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [1], 'recovery': []},
        },
        '521886': {
            'data_loc': r'C:\Users\lesliec\OneDrive - Allen Institute\data\mouse521886\estim1_2020-07-16_13-37-02\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [1], 'recovery': [2]},
        },
        '521887': {
            'data_loc': r'C:\Users\lesliec\OneDrive - Allen Institute\data\mouse521887\estim1_2020-07-30_11-25-05\experiment1\recordingfull',
            'states': {'awake': [0, 1], 'anesthetized': [2, 3, 4], 'recovery': [5, 6, 7]},
        },
        '543393': {
            'data_loc': r'F:\EEG_exp\mouse543393\estim1_2020-08-20_13-24-53\experiment1\recording1',
            'states': {'awake': [0, 1, 2], 'anesthetized': [3, 4, 5], 'recovery': [6, 7, 8]},
        },
        '543394': {
            'data_loc': r'F:\EEG_exp\mouse543394\estim1_2020-08-27_14-32-00\experiment1\recording1',
            'states': {'awake': [0, 1, 2], 'anesthetized': [3, 4, 5], 'recovery': [6, 7, 8]},
        },
#         '543395': { ### issues with large noise starting in anesthetized state ###
#             'data_loc': r'F:\EEG_exp\mouse543395\estim1_2020-09-03_13-19-42\experiment1\recording1',
#             'states': {'awake': [0, 1, 2], 'anesthetized': [3, 4, 5, 6], 'recovery': [7, 8, 9]},
#         },
#         '543396': { ### too few resting trials ###
#             'data_loc': r'F:\EEG_exp\mouse543396\estim_vis1_2020-09-18_12-04-46\experiment1\recording1',
#             'states': {'awake': [0], 'anesthetized': [1], 'recovery': [2]},
#         },
#         '571618': { ### too many bad chs ###
#             'data_loc': r'F:\EEG_exp\mouse571618\estim1_2021-04-29_12-28-54\experiment1\recording1',
#             'states': {'awake': [0], 'anesthetized': [1], 'recovery': []},
#         },
        '575102': {
            'data_loc': r'F:\EEG_exp\mouse575102\estim_vis_2021-06-03_11-25-01\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [1], 'recovery': [2, 3]},
        },
        '571619': { ### stim in ANT MOs at 2 depths ###
            'data_loc': r'F:\EEG_exp\mouse571619\estim2_2021-03-19_10-09-01\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [3], 'recovery': []},
        },
    },
    'MOs_deep': {
#         '551400': { ### bad EEG signals ##
#             'data_loc': r'F:\EEG_exp\mouse551400\estim_vis_2021-01-22_11-07-12\experiment1\recording1',
#             'states': {'awake': [0], 'anesthetized': [1], 'recovery': [2, 3]},
#         },
        '546655': { ### stim was actually quite deep based on histology ###
            'data_loc': r'F:\EEG_exp\mouse546655\estim_vis_2020-10-23_11-01-14\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [1], 'recovery': [2, 3]},
        },
        '551399': {
            'data_loc': r'F:\EEG_exp\mouse551399\estim_2021-01-28_13-59-09\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [1], 'recovery': [2, 3]},
        },
        '551397': {
            'data_loc': r'F:\EEG_exp\mouse551397\estim_vis_2021-02-11_10-45-23\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [1], 'recovery': [2, 3]},
        },
        '569062': {
            'data_loc': r'F:\EEG_exp\mouse569062\estim_vis_2021-02-18_11-17-51\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [1], 'recovery': [2]},
        },
        '569068': {
            'data_loc': r'F:\EEG_exp\mouse569068\estim_vis_2021-03-04_10-51-38\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [1], 'recovery': [2, 3]},
        },
        '569069': {
            'data_loc': r'F:\EEG_exp\mouse569069\estim_vis2_2021-03-12_10-52-44\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [1], 'recovery': [2, 3]},
        },
        '569064': {
            'data_loc': r'F:\EEG_exp\mouse569064\estim_vis_2021-04-08_10-28-24\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [1], 'recovery': [2, 3]},
        },
        '569073': {
            'data_loc': r'F:\EEG_exp\mouse569073\estim_vis_2021-04-15_10-27-22\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [1], 'recovery': [2, 3]},
        },
        '571619': { ### stim in ANT MOs at 2 depths ###
            'data_loc': r'F:\EEG_exp\mouse571619\estim2_2021-03-19_10-09-01\experiment1\recording1',
            'states': {'awake': [1], 'anesthetized': [2], 'recovery': []},
        },
        '569070': { ### stim in ANT MOs at 2 depths, deep stim is at 800, 1200 depth not included ###
            'data_loc': r'F:\EEG_exp\mouse569070\estim1_2021-04-01_10-27-33\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [1], 'recovery': []},
        },
    },
    'SSp_superficial': {
        '571620': {
            'data_loc': r'F:\EEG_exp\mouse571620\estim_vis_2021-05-13_11-33-47\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [3], 'recovery': []},
        },
#         '582387': { ### too many bad chs at both depths ###
#             'data_loc': r'F:\EEG_exp\mouse582387\estim_vis_2021-07-22_09-59-47\experiment1\recording1',
#             'states': {'awake': [0], 'anesthetized': [3], 'recovery': []},
#         },
        '586466': {
            'data_loc': r'F:\EEG_exp\mouse586466\estim_vis_2021-07-29_10-37-41\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [3], 'recovery': []},
        },
#         '586468': { ### too many bad chs at superficial depth ###
#             'data_loc': r'F:\EEG_exp\mouse586468\estim_vis_2021-08-19_11-36-53\experiment1\recording1',
#             'states': {'awake': [0], 'anesthetized': [3], 'recovery': []},
#         },
        '590479': { ### more bad chs at superficial depth ###
            'data_loc': r'F:\EEG_exp\mouse590479\estim_2021-09-21_17-24-07\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [3], 'recovery': []},
            'bad_chs': [0,4,5,6,7,8,9,10,11,12,13,14],
        },
        '590480': {
            'data_loc': r'F:\EEG_exp\mouse590480\estim_vis_2021-11-11_10-54-19\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [3], 'recovery': []},
        },
#         '590481': { ### more bad chs at superficial depth, large lesion in SSp ###
#             'data_loc': r'F:\EEG_exp\mouse590481\estim_vis_2021-11-18_10-52-12\experiment1\recording1',
#             'states': {'awake': [0], 'anesthetized': [3], 'recovery': []},
#             'bad_chs': [0,1,4,5,7,8,10,13,14],
#         },
        '599017': { ### more bad chs at superficial depth ###
            'data_loc': r'F:\EEG_exp\mouse599017\estim_vis_2022-02-10_10-53-32\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [3], 'recovery': []},
            'bad_chs': [0,3,4,7,8,9,10,11,13],
        },
    },
    'SSp_deep': {
        '569073': {
            'data_loc': r'F:\EEG_exp\mouse569073\estim_2021-04-16_10-42-44\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [1], 'recovery': []},
        },
        '569072': {
            'data_loc': r'F:\EEG_exp\mouse569072\estim_vis_2021-04-22_10-26-58\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [1], 'recovery': []},
        },
        '571620': {
            'data_loc': r'F:\EEG_exp\mouse571620\estim_vis_2021-05-13_11-33-47\experiment1\recording1',
            'states': {'awake': [1], 'anesthetized': [2], 'recovery': []},
        },
#         '582387': { ### too many bad chs at both depths ###
#             'data_loc': r'F:\EEG_exp\mouse582387\estim_vis_2021-07-22_09-59-47\experiment1\recording1',
#             'states': {'awake': [1], 'anesthetized': [2], 'recovery': []},
#         },
        '586466': {
            'data_loc': r'F:\EEG_exp\mouse586466\estim_vis_2021-07-29_10-37-41\experiment1\recording1',
            'states': {'awake': [1], 'anesthetized': [2], 'recovery': []},
        },
        '586468': { ### stim in VISa ###
            'data_loc': r'F:\EEG_exp\mouse586468\estim_vis_2021-08-19_11-36-53\experiment1\recording1',
            'states': {'awake': [1], 'anesthetized': [2], 'recovery': []},
        },
        '590479': { ### more bad chs at superficial depth ###
            'data_loc': r'F:\EEG_exp\mouse590479\estim_2021-09-21_17-24-07\experiment1\recording1',
            'states': {'awake': [1], 'anesthetized': [2], 'recovery': []},
        },
        '590480': {
            'data_loc': r'F:\EEG_exp\mouse590480\estim_vis_2021-11-11_10-54-19\experiment1\recording1',
            'states': {'awake': [1], 'anesthetized': [2], 'recovery': []},
        },
#         '590481': { ### more bad chs at superficial depth, large lesion in SSp ###
#             'data_loc': r'F:\EEG_exp\mouse590481\estim_vis_2021-11-18_10-52-12\experiment1\recording1',
#             'states': {'awake': [1], 'anesthetized': [2], 'recovery': []},
#             'bad_chs': [8,10,13,14],
#         },
        '599017': { ### more bad chs at superficial depth ###
            'data_loc': r'F:\EEG_exp\mouse599017\estim_vis_2022-02-10_10-53-32\experiment1\recording1',
            'states': {'awake': [1], 'anesthetized': [2], 'recovery': []},
        },
    },
}

In [8]:
## Save the data to .json file ##
with open(r'C:\Users\lesliec\OneDrive - Allen Institute\data\all_iso_subjects.json', 'w') as outfile:
    json.dump(all_subjects, outfile, indent = 4, separators = (',', ': '))

### Master list of subjects for zap&zip 1st manuscript - subjects with probes and locations only

In [7]:
all_subjects = {
    'MOs_superficial': {
#         '543396': { ### too few resting trials ###
#             'data_loc': r'F:\EEG_exp\mouse543396\estim_vis1_2020-09-18_12-04-46\experiment1\recording1',
#             'states': {'awake': [0], 'anesthetized': [1], 'recovery': [2]},
#         },
#         '571618': { ### too many bad chs ###
#             'data_loc': r'F:\EEG_exp\mouse571618\estim1_2021-04-29_12-28-54\experiment1\recording1',
#             'states': {'awake': [0], 'anesthetized': [1], 'recovery': []},
#         },
        '575102': {
            'data_loc': r'F:\EEG_exp\mouse575102\estim_vis_2021-06-03_11-25-01\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [1], 'recovery': [2, 3]},
        },
        '571619': { ### stim in ANT MOs at 2 depths ###
            'data_loc': r'F:\EEG_exp\mouse571619\estim2_2021-03-19_10-09-01\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [3], 'recovery': []},
        },
    },
    'MOs_deep': {
#         '551400': { ### bad EEG signals ##
#             'data_loc': r'F:\EEG_exp\mouse551400\estim_vis_2021-01-22_11-07-12\experiment1\recording1',
#             'states': {'awake': [0], 'anesthetized': [1], 'recovery': [2, 3]},
#         },
        '546655': {
            'data_loc': r'F:\EEG_exp\mouse546655\estim_vis_2020-10-23_11-01-14\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [1], 'recovery': [2, 3]},
        },
        '551399': {
            'data_loc': r'F:\EEG_exp\mouse551399\estim_2021-01-28_13-59-09\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [1], 'recovery': [2, 3]},
        },
        '551397': {
            'data_loc': r'F:\EEG_exp\mouse551397\estim_vis_2021-02-11_10-45-23\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [1], 'recovery': [2, 3]},
        },
        '569062': {
            'data_loc': r'F:\EEG_exp\mouse569062\estim_vis_2021-02-18_11-17-51\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [1], 'recovery': [2]},
        },
        '569068': {
            'data_loc': r'F:\EEG_exp\mouse569068\estim_vis_2021-03-04_10-51-38\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [1], 'recovery': [2, 3]},
        },
        '569069': {
            'data_loc': r'F:\EEG_exp\mouse569069\estim_vis2_2021-03-12_10-52-44\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [1], 'recovery': [2, 3]},
        },
        '569064': {
            'data_loc': r'F:\EEG_exp\mouse569064\estim_vis_2021-04-08_10-28-24\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [1], 'recovery': [2, 3]},
        },
        '569073': {
            'data_loc': r'F:\EEG_exp\mouse569073\estim_vis_2021-04-15_10-27-22\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [1], 'recovery': [2, 3]},
        },
        '571619': { ### stim in ANT MOs at 2 depths ###
            'data_loc': r'F:\EEG_exp\mouse571619\estim2_2021-03-19_10-09-01\experiment1\recording1',
            'states': {'awake': [1], 'anesthetized': [2], 'recovery': []},
        },
#         '569070': { ### NO PROBE LOCS - stim in ANT MOs at 2 depths, deep stim is at 800, 1200 depth not included ###
#             'data_loc': r'F:\EEG_exp\mouse569070\estim1_2021-04-01_10-27-33\experiment1\recording1',
#             'states': {'awake': [0], 'anesthetized': [1], 'recovery': []},
#         },
    },
    'SSp_superficial': {
        '571620': {
            'data_loc': r'F:\EEG_exp\mouse571620\estim_vis_2021-05-13_11-33-47\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [3], 'recovery': []},
        },
#         '582387': { ### too many bad chs at both depths ###
#             'data_loc': r'F:\EEG_exp\mouse582387\estim_vis_2021-07-22_09-59-47\experiment1\recording1',
#             'states': {'awake': [0], 'anesthetized': [3], 'recovery': []},
#         },
        '586466': {
            'data_loc': r'F:\EEG_exp\mouse586466\estim_vis_2021-07-29_10-37-41\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [3], 'recovery': []},
        },
#         '586468': { ### too many bad chs at superficial depth ###
#             'data_loc': r'F:\EEG_exp\mouse586468\estim_vis_2021-08-19_11-36-53\experiment1\recording1',
#             'states': {'awake': [0], 'anesthetized': [3], 'recovery': []},
#         },
        '590479': { ### more bad chs at superficial depth ###
            'data_loc': r'F:\EEG_exp\mouse590479\estim_2021-09-21_17-24-07\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [3], 'recovery': []},
            'bad_chs': [0,4,5,6,7,8,9,10,11,12,13,14],
        },
        '590480': {
            'data_loc': r'F:\EEG_exp\mouse590480\estim_vis_2021-11-11_10-54-19\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [3], 'recovery': []},
        },
#         '590481': { ### more bad chs at superficial depth, large lesion in SSp ###
#             'data_loc': r'F:\EEG_exp\mouse590481\estim_vis_2021-11-18_10-52-12\experiment1\recording1',
#             'states': {'awake': [0], 'anesthetized': [3], 'recovery': []},
#             'bad_chs': [0,1,4,5,7,8,10,13,14],
#         },
        '599017': { ### more bad chs at superficial depth ###
            'data_loc': r'F:\EEG_exp\mouse599017\estim_vis_2022-02-10_10-53-32\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [3], 'recovery': []},
            'bad_chs': [0,3,4,7,8,9,10,11,13],
        },
    },
    'SSp_deep': {
        '569073': {
            'data_loc': r'F:\EEG_exp\mouse569073\estim_2021-04-16_10-42-44\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [1], 'recovery': []},
        },
        '569072': {
            'data_loc': r'F:\EEG_exp\mouse569072\estim_vis_2021-04-22_10-26-58\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [1], 'recovery': []},
        },
        '571620': {
            'data_loc': r'F:\EEG_exp\mouse571620\estim_vis_2021-05-13_11-33-47\experiment1\recording1',
            'states': {'awake': [1], 'anesthetized': [2], 'recovery': []},
        },
#         '582387': { ### too many bad chs at both depths ###
#             'data_loc': r'F:\EEG_exp\mouse582387\estim_vis_2021-07-22_09-59-47\experiment1\recording1',
#             'states': {'awake': [1], 'anesthetized': [2], 'recovery': []},
#         },
        '586466': {
            'data_loc': r'F:\EEG_exp\mouse586466\estim_vis_2021-07-29_10-37-41\experiment1\recording1',
            'states': {'awake': [1], 'anesthetized': [2], 'recovery': []},
        },
        '586468': { ### stim in VISa ###
            'data_loc': r'F:\EEG_exp\mouse586468\estim_vis_2021-08-19_11-36-53\experiment1\recording1',
            'states': {'awake': [1], 'anesthetized': [2], 'recovery': []},
        },
        '590479': { ### more bad chs at superficial depth ###
            'data_loc': r'F:\EEG_exp\mouse590479\estim_2021-09-21_17-24-07\experiment1\recording1',
            'states': {'awake': [1], 'anesthetized': [2], 'recovery': []},
        },
        '590480': {
            'data_loc': r'F:\EEG_exp\mouse590480\estim_vis_2021-11-11_10-54-19\experiment1\recording1',
            'states': {'awake': [1], 'anesthetized': [2], 'recovery': []},
        },
#         '590481': { ### more bad chs at superficial depth, large lesion in SSp ###
#             'data_loc': r'F:\EEG_exp\mouse590481\estim_vis_2021-11-18_10-52-12\experiment1\recording1',
#             'states': {'awake': [1], 'anesthetized': [2], 'recovery': []},
#             'bad_chs': [8,10,13,14],
#         },
        '599017': { ### more bad chs at superficial depth ###
            'data_loc': r'F:\EEG_exp\mouse599017\estim_vis_2022-02-10_10-53-32\experiment1\recording1',
            'states': {'awake': [1], 'anesthetized': [2], 'recovery': []},
        },
    },
}

In [8]:
## Save the data to .json file ##
with open(r'C:\Users\lesliec\OneDrive - Allen Institute\data\all_iso_subjects_wPROBES.json', 'w') as outfile:
    json.dump(all_subjects, outfile, indent = 4, separators = (',', ': '))

### List of subjects with probes and clean CSD data

In [10]:
## Save the data to .json file ##
with open(r'C:\Users\lesliec\OneDrive - Allen Institute\data\all_CSD_subjects.json', 'w') as outfile:
    json.dump(all_subjects, outfile, indent = 4, separators = (',', ': '))

### TEST list of subjects for zap&zip 1st manuscript - subjects with probes and locations only

In [6]:
all_subjects = {
    'MOs_superficial': {
        '575102': {
            'data_loc': r'F:\EEG_exp\mouse575102\estim_vis_2021-06-03_11-25-01\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [1], 'recovery': [2, 3]},
        },
    },
    'MOs_deep': {
        '551397': {
            'data_loc': r'F:\EEG_exp\mouse551397\estim_vis_2021-02-11_10-45-23\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [1], 'recovery': [2, 3]},
        },
        '569062': {
            'data_loc': r'F:\EEG_exp\mouse569062\estim_vis_2021-02-18_11-17-51\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [1], 'recovery': [2]},
        },
}

### TEST list of subject with ctx responsive files

In [7]:
all_subjects = {
    'MOs_superficial': {
        '571619': { ### stim in ANT MOs at 2 depths ### ## Removed for faster testing ##
            'data_loc': r'F:\EEG_exp\mouse571619\estim2_2021-03-19_10-09-01\experiment1\recording1',
            'states': {'awake': [0], 'anesthetized': [3], 'recovery': []},
        },
    },
    'MOs_deep': {
        '571619': { ### stim in ANT MOs at 2 depths ### ## Removed for faster testing ##
            'data_loc': r'F:\EEG_exp\mouse571619\estim2_2021-03-19_10-09-01\experiment1\recording1',
            'states': {'awake': [1], 'anesthetized': [2], 'recovery': []},
        },
    },
}

### All EEG subjects

In [10]:
all_subjects = {
    '521885': r'C:\Users\lesliec\OneDrive - Allen Institute\data\mouse521885\estim1_2020-07-09_14-23-49\experiment1\recording1',
    '521886': r'C:\Users\lesliec\OneDrive - Allen Institute\data\mouse521886\estim1_2020-07-16_13-37-02\experiment1\recording1',
    '521887': r'C:\Users\lesliec\OneDrive - Allen Institute\data\mouse521887\estim1_2020-07-30_11-25-05\experiment1\recordingfull',
    '543393': r'F:\EEG_exp\mouse543393\estim1_2020-08-20_13-24-53\experiment1\recording1',
    '543394': r'F:\EEG_exp\mouse543394\estim1_2020-08-27_14-32-00\experiment1\recording1',
    '543395': r'F:\EEG_exp\mouse543395\estim1_2020-09-03_13-19-42\experiment1\recording1', ### large noise ###
    '543396': r'F:\EEG_exp\mouse543396\estim_vis1_2020-09-18_12-04-46\experiment1\recording1', ### too few resting trials ###
    '546655': r'F:\EEG_exp\mouse546655\estim_vis_2020-10-23_11-01-14\experiment1\recording1',
    '551400': r'F:\EEG_exp\mouse551400\estim_vis_2021-01-22_11-07-12\experiment1\recording1', ### bad EEG signals ##
    '551399': r'F:\EEG_exp\mouse551399\estim_2021-01-28_13-59-09\experiment1\recording1',
    '551397': r'F:\EEG_exp\mouse551397\estim_vis_2021-02-11_10-45-23\experiment1\recording1',
    '569062': r'F:\EEG_exp\mouse569062\estim_vis_2021-02-18_11-17-51\experiment1\recording1',
    '569068': r'F:\EEG_exp\mouse569068\estim_vis_2021-03-04_10-51-38\experiment1\recording1',
    '569069': r'F:\EEG_exp\mouse569069\estim_vis2_2021-03-12_10-52-44\experiment1\recording1',
    '569064': r'F:\EEG_exp\mouse569064\estim_vis_2021-04-08_10-28-24\experiment1\recording1',
    '569073a': r'F:\EEG_exp\mouse569073\estim_vis_2021-04-15_10-27-22\experiment1\recording1',
    '569073b': r'F:\EEG_exp\mouse569073\estim_2021-04-16_10-42-44\experiment1\recording1',
    '571618': r'F:\EEG_exp\mouse571618\estim1_2021-04-29_12-28-54\experiment1\recording1', ### too many bad chs ###
    '575102': r'F:\EEG_exp\mouse575102\estim_vis_2021-06-03_11-25-01\experiment1\recording1',
    '575100': r'F:\EEG_exp\mouse575100\estim_vis_2021-06-10_11-39-06\experiment1\recording1',
#     '575101': r'', ### apparently no EEG or unit response, too much cortical damage? ###
    '571619a': r'F:\EEG_exp\mouse571619\estim1_2021-03-18_11-41-26\experiment1\recording1',
    '571619b': r'F:\EEG_exp\mouse571619\estim2_2021-03-19_10-09-01\experiment1\recording1',
    '569070a': r'F:\EEG_exp\mouse569070\estim1_2021-04-01_10-27-33\experiment1\recording1', ### antMOs ###
    '569070b': r'F:\EEG_exp\mouse569070\estim2_2021-04-02_10-32-01\experiment1\recording1', ### antMOs ###
    '569072': r'F:\EEG_exp\mouse569072\estim_vis_2021-04-22_10-26-58\experiment1\recording1',
    '571620': r'F:\EEG_exp\mouse571620\estim_vis_2021-05-13_11-33-47\experiment1\recording1',
    '582387': r'F:\EEG_exp\mouse582387\estim_vis_2021-07-22_09-59-47\experiment1\recording1', ### too many bad chs ###
    '586466': r'F:\EEG_exp\mouse586466\estim_vis_2021-07-29_10-37-41\experiment1\recording1',
    '586468a': r'F:\EEG_exp\mouse586468\estim_vis_2021-08-19_11-36-53\experiment1\recording1', ### too many bad chs ###
    '586468b': r'F:\EEG_exp\mouse586468\estim_2021-08-20_11-34-54\experiment1\recording1', ### 100 um estim ###
    '590479': r'F:\EEG_exp\mouse590479\estim_2021-09-21_17-24-07\experiment1\recording1',
    '590480': r'F:\EEG_exp\mouse590480\estim_vis_2021-11-11_10-54-19\experiment1\recording1',
    '590481': r'F:\EEG_exp\mouse590481\estim_vis_2021-11-18_10-52-12\experiment1\recording1', ### large lesion in SSp ###
    '599017': r'F:\EEG_exp\mouse599017\estim_vis_2022-02-10_10-53-32\experiment1\recording1',
}

### All subjects with probes

In [7]:
all_subjects = {
#     '543393': r'F:\EEG_exp\mouse543393\estim1_2020-08-20_13-24-53\experiment1\recording1',
#     '543394': r'F:\EEG_exp\mouse543394\estim1_2020-08-27_14-32-00\experiment1\recording1',
#     '543395': r'F:\EEG_exp\mouse543395\estim1_2020-09-03_13-19-42\experiment1\recording1', ### large noise ###
#     '543396': r'F:\EEG_exp\mouse543396\estim_vis1_2020-09-18_12-04-46\experiment1\recording1', ### too few resting trials ###
#     '546655': r'F:\EEG_exp\mouse546655\estim_vis_2020-10-23_11-01-14\experiment1\recording1',
#     '551400': r'F:\EEG_exp\mouse551400\estim_vis_2021-01-22_11-07-12\experiment1\recording1', ### bad EEG signals ##
#     '551399': r'F:\EEG_exp\mouse551399\estim_2021-01-28_13-59-09\experiment1\recording1',
#     '551397': r'F:\EEG_exp\mouse551397\estim_vis_2021-02-11_10-45-23\experiment1\recording1',
#     '569062': r'F:\EEG_exp\mouse569062\estim_vis_2021-02-18_11-17-51\experiment1\recording1',
#     '569068': r'F:\EEG_exp\mouse569068\estim_vis_2021-03-04_10-51-38\experiment1\recording1',
#     '569069': r'F:\EEG_exp\mouse569069\estim_vis2_2021-03-12_10-52-44\experiment1\recording1',
#     '569064': r'F:\EEG_exp\mouse569064\estim_vis_2021-04-08_10-28-24\experiment1\recording1',
    '569073a': r'F:\EEG_exp\mouse569073\estim_vis_2021-04-15_10-27-22\experiment1\recording1',
#     '569073b': r'F:\EEG_exp\mouse569073\estim_2021-04-16_10-42-44\experiment1\recording1',
#     '571618': r'F:\EEG_exp\mouse571618\estim1_2021-04-29_12-28-54\experiment1\recording1', ### too many bad chs ###
#     '575102': r'F:\EEG_exp\mouse575102\estim_vis_2021-06-03_11-25-01\experiment1\recording1',
#     '575100': r'F:\EEG_exp\mouse575100\estim_vis_2021-06-10_11-39-06\experiment1\recording1',
#     '571619a': r'F:\EEG_exp\mouse571619\estim1_2021-03-18_11-41-26\experiment1\recording1', ## stim in HPC ##
    '571619b': r'F:\EEG_exp\mouse571619\estim2_2021-03-19_10-09-01\experiment1\recording1',
#     '569070a': r'F:\EEG_exp\mouse569070\estim1_2021-04-01_10-27-33\experiment1\recording1', ### antMOs ###
#     '569070b': r'F:\EEG_exp\mouse569070\estim2_2021-04-02_10-32-01\experiment1\recording1', ### antMOs ###
#     '569072': r'F:\EEG_exp\mouse569072\estim_vis_2021-04-22_10-26-58\experiment1\recording1',
#     '571620': r'F:\EEG_exp\mouse571620\estim_vis_2021-05-13_11-33-47\experiment1\recording1',
#     '582387': r'F:\EEG_exp\mouse582387\estim_vis_2021-07-22_09-59-47\experiment1\recording1', ### too many bad chs ###
#     '586466': r'F:\EEG_exp\mouse586466\estim_vis_2021-07-29_10-37-41\experiment1\recording1',
#     '586468a': r'F:\EEG_exp\mouse586468\estim_vis_2021-08-19_11-36-53\experiment1\recording1', ### too many bad chs ###
#     '586468b': r'F:\EEG_exp\mouse586468\estim_2021-08-20_11-34-54\experiment1\recording1', ### 100 um estim ###
#     '590479': r'F:\EEG_exp\mouse590479\estim_2021-09-21_17-24-07\experiment1\recording1',
#     '590480': r'F:\EEG_exp\mouse590480\estim_vis_2021-11-11_10-54-19\experiment1\recording1',
#     '590481': r'F:\EEG_exp\mouse590481\estim_vis_2021-11-18_10-52-12\experiment1\recording1', ### large lesion in SSp ###
#     '599017': r'F:\EEG_exp\mouse599017\estim_vis_2022-02-10_10-53-32\experiment1\recording1',
}

### All subjects for EEG/LFP grant

In [6]:
all_subjects = {
    '551397': {
        'data_loc': r'F:\EEG_exp\mouse551397\estim_vis_2021-02-11_10-45-23\experiment1\recording1',
        'states': {'awake': [0], 'isoflurane': [1], 'recovery': [2, 3]},
    },
#     '569062': {
#         'data_loc': r'F:\EEG_exp\mouse569062\estim_vis_2021-02-18_11-17-51\experiment1\recording1',
#         'states': {'awake': [0], 'isoflurane': [1], 'recovery': [2]},
#     },
#     '569068': { ## ref screws not through skull ##
#         'data_loc': r'F:\EEG_exp\mouse569068\estim_vis_2021-03-04_10-51-38\experiment1\recording1',
#         'states': {'awake': [0], 'isoflurane': [1], 'recovery': [2, 3]},
#     },
    '569069': {
        'data_loc': r'F:\EEG_exp\mouse569069\estim_vis2_2021-03-12_10-52-44\experiment1\recording1',
        'states': {'awake': [0], 'isoflurane': [1], 'recovery': [2, 3]},
    },
#     '569064': {
#         'data_loc': r'F:\EEG_exp\mouse569064\estim_vis_2021-04-08_10-28-24\experiment1\recording1',
#         'states': {'awake': [0], 'isoflurane': [1], 'recovery': [2, 3]},
#     },
#     '569073': { ## ref screws not through skull ##
#         'data_loc': r'F:\EEG_exp\mouse569073\estim_vis_2021-04-15_10-27-22\experiment1\recording1',
#         'states': {'awake': [0], 'isoflurane': [1], 'recovery': [2, 3]},
#     },
#     '575102': {
#         'data_loc': r'F:\EEG_exp\mouse575102\estim_vis_2021-06-03_11-25-01\experiment1\recording1',
#         'states': {'awake': [0], 'isoflurane': [1], 'recovery': [2, 3]},
#     },
    '575100': {
        'data_loc': r'F:\EEG_exp\mouse575100\estim_vis_2021-06-10_11-39-06\experiment1\recording1',
        'states': {'awake': [0], 'isoflurane': [1], 'recovery': []},
    },
#     '569072': {
#         'data_loc': r'F:\EEG_exp\mouse569072\estim_vis_2021-04-22_10-26-58\experiment1\recording1',
#         'states': {'awake': [0], 'isoflurane': [1], 'recovery': []},
#     },
    '571620': {
        'data_loc': r'F:\EEG_exp\mouse571620\estim_vis_2021-05-13_11-33-47\experiment1\recording1',
        'states': {'awake': [0], 'isoflurane': [1], 'recovery': []},
    },
#     '582387': { ### many bad chs for estim ###
#         'data_loc': r'F:\EEG_exp\mouse582387\estim_vis_2021-07-22_09-59-47\experiment1\recording1',
#         'states': {'awake': [0], 'isoflurane': [1], 'recovery': []},
#     },
    '586466': {
        'data_loc': r'F:\EEG_exp\mouse586466\estim_vis_2021-07-29_10-37-41\experiment1\recording1',
        'states': {'awake': [0], 'isoflurane': [1], 'recovery': []},
    },
#     '586468': {  ## ref screws not through skull ##
#         'data_loc': r'F:\EEG_exp\mouse586468\estim_vis_2021-08-19_11-36-53\experiment1\recording1',
#         'states': {'awake': [0], 'isoflurane': [1], 'recovery': []},
#     },
    '590480a': {
        'data_loc': r'F:\EEG_exp\mouse590480\estim_vis_2021-11-11_10-54-19\experiment1\recording1',
        'states': {'awake': [0], 'isoflurane': [1], 'recovery': []},
    },
#     '590480b': {
#         'data_loc': r'F:\EEG_exp\mouse590480\urethane_vis_2021-11-12_11-14-43\experiment1\recording1',
#         'states': {'urethane': [0], 'burst_suppression': [1], 'recovery': []},
#     },
#     '590481a': { ### something appears to be wrong with the EEG signals ###
#         'data_loc': r'F:\EEG_exp\mouse590481\estim_vis_2021-11-18_10-52-12\experiment1\recording1',
#         'states': {'awake': [0], 'isoflurane': [1], 'recovery': []},
#     },
#     '590481b': {
#         'data_loc': r'F:\EEG_exp\mouse590481\urethane_vis_2021-11-19_10-59-04\experiment1\recording1',
#         'states': {'urethane': [0], 'burst_suppression': [1], 'recovery': []},
#     },
}

### Load experiments

Load the .json file containing multi-subject dictionary